<a href="https://colab.research.google.com/github/Abdansb/NoCopyZone/blob/main/NoCopyZone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To-Do

*   Gather all selenium result in one list
*   learn regex
*   fix result URL's with prefix "/"
*   scrape via HTTP traffic
*   scrape via source (js, html, and other files)

<div>
<img src="https://drive.usercontent.google.com/download?id=1kh0VlEu1NDOVqXyYKHzFviRMCyIOwpgZ&export=download&authuser=0&confirm=t&uuid=39cde4b1-3c0f-49a2-9624-162fc043d877&at=APZUnTXs12dtKQbVEqDxjjwtmEw2:1702054613486" width="400"/>
</div>





# Install Dependecies

###Bash scripting, accessing google colab terminal. Install python packages to their OS

In [ ]:
! pip install requests bs4
! pip install selenium
! apt-get update
! apt install chromium-chromedriver

# Using Selenium & Regular Expression

*   Weekend kulik TDD Test-driven development
*   Currently scrape all links in "a" tag
*   PR! some websites use hidden js






In [ ]:
# Import and configure chrome browser

import re
from selenium.webdriver.common.by import By
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #headless means no graphical user interface
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
driver.get('https://livescore90bola.com/')
# Get element by tag: <a> or <div> or <script>

# combinations of getting URL's

#links = driver.find_elements(By.TAG_NAME, 'a')
links = driver.find_elements(By.TAG_NAME, 'div')
#links = driver.find_elements(By.TAG_NAME, 'script')

# Iterate over links list
for link in links:

  # Get raw html (ambil satu baris)
  html_string = link.get_attribute('outerHTML')

  # Capture string after "href="
  url_match = re.search(r'href="(.*?)"', html_string)

  # Capture string after "src="
  #url_match = re.search(r'src="(.*?)"', html_string)

  if url_match:
    url = url_match.group(1)

  print(url)






/
indonesia.html
indonesia.html
indonesia.html
indonesia.html
indonesia.html
indonesia.html
indonesia.html
indonesia.html
indonesia.html
javascript:window.scroll(0,0);


# Gak jadi
# Using BeautifulSoup

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# import time

source_code = requests.get("https://livescore90bola.com/")
soup = BeautifulSoup(source_code.content, "lxml")
data = []
links = []


def remove_duplicates(l):  # remove duplicates and unURL string
    for item in l:
        match = re.search("(?P<url>https?://[^\s]+)", item)
        if match is not None:
            links.append((match.group("url")))


for link in soup.find_all("a", href=True):
    data.append(str(link.get("href")))
flag = True
remove_duplicates(data)
lim = 50
while flag:
    try:
        for link in links:
            for j in soup.find_all("a", href=True):
                temp = []
                source_code = requests.get(link)
                soup = BeautifulSoup(source_code.content, "lxml")
                temp.append(str(j.get("href")))
                remove_duplicates(temp)
                print(temp)

                if len(links) > lim:  # set limitation to number of URLs
                    break
            if len(links) > lim:
                break
        if len(links) > lim:
            break
    except Exception as e:
        print(e)
        if len(links) > lim:
            break

for url in links:
    print(url)


['/']
['https://www.facebook.com/90bola.io/']
['https://www.youtube.com/channel/UCSMkjRpTQfwM601z3KSzg0g']
['https://www.instagram.com/90bola.io/']
['/']
['/']
['basket.html']
['bisbol.html']
['kalkulator.html']
['/']
['https://www.facebook.com/90bola.io/']
['jadwal.html']
['hasil.html']
['1x2.html']
['https://nowgoal5.io/']
['bursa.html']
['indonesia.html']
['champion.html']
['eropa.html']
['inggris.html']
['italia.html']
['spanyol.html']
['jerman.html']
['perancis.html']
['portugal.html']
['javascript:window.scroll(0,0);']
['tentang-kami.html']
['#']
['#fb']
['petagratis.html']
['sitemap.xml']
['#']
['petagratis.html']
['#']
['#']
['#']
['https://livescore90bola.com/']
['tentang-kami.html']
['#']
['#']
['https://855group.page.link/fifa855']
['tentang-kami.html']
['/']
['/']
['']
['']
['']
['#']
['https://www.facebook.com/recover/initiate/?ars=facebook_login']
['/r.php?next=https%3A%2F%2Fwww.facebook.com%2F90bola.io%2F&locale=en_US&display=page']
['https://es-la.facebook.com/login/?ne